In [9]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

import bs4
import re

## For a player

In [10]:
options = webdriver.EdgeOptions()
# options.add_argument('headless')
options.add_argument('inprivate')
driver = webdriver.Edge(options= options)

In [11]:
driver.get('https://qlstats.net/player/63562')

### Cookie screen

In [12]:
def is_cookie_screen(driver : selenium.webdriver):
    '''A function to check if the given webpage is the 'accept cookies' screen.
    Regex matches the body of the '''
    element = driver.find_element(By.TAG_NAME, value = 'body')
    cookie_screen = re.compile(r'.*(To continue using qlstats, you need to agree to the use of cookies.\nAgree).*')
    if cookie_screen.search(element.text) is None:
        return False
    return True


In [13]:
#press the button
if is_cookie_screen(driver):
    try:
        button = driver.find_element(By.TAG_NAME, 'button')
        button.click()
    except:
        "Cookies could not be accepted, please recheck"

### Scraping the player page

In [14]:
from bs4 import BeautifulSoup

In [15]:
soup = BeautifulSoup(driver.page_source, 'lxml')


In [16]:
# finding player name
player_name = soup.select_one('h2').text
print('Player name:\n{}'.format(player_name))

#for unavailable IDs:
if re.match(r"Sorry, that player wasn't found!", player_name):
    print("Player not found, can't proceed")

else:
    pass
    #TODO Scrape here

Player name:
A S U R A


### ELSE tag here onwards

- p_tab_list
- player_info
- games_played
- type_stats

dictionaries

In [17]:
# parse information from the p tab
p_tab_text = soup.select_one('p').text
p_tab_list = list(map(str.strip, p_tab_text.split("\n")))
p_tab_list = [x for x in p_tab_list if x != '']

p_tab_list

['Region: Asia',
 'Player ID: 63562',
 'Steam ID: 76561197968261050',
 'Joined: 2016-02-15   22:51:44',
 'Status: active']

#### Split and store in dictionary

In [18]:
player_info = {'name' : player_name}
for x in p_tab_list:
    elements = x.split(": ")
    player_info[elements[0]] = elements[1]

player_info

{'name': 'A S U R A',
 'Region': 'Asia',
 'Player ID': '63562',
 'Steam ID': '76561197968261050',
 'Joined': '2016-02-15   22:51:44',
 'Status': 'active'}

In [19]:
# press the overall button on the player page
button = driver.find_element(By.CLASS_NAME, 'tab-overall')
button.click()

In [41]:
num = '(7340)'
int(num[1:-1])

7340

In [47]:
games_played = {}

gametypes = soup.find('ul', id = 'gbtab').text
gametypes = [x.strip() for x in gametypes.split('\n') if x != '']

for i in range(len(gametypes)//2):
    t, n = gametypes[i*2], gametypes[2*i+1][1:-1]
    games_played[t] = int(n)

games_played

{'overall': 7538,
 'ca': 6677,
 'duel': 731,
 'ffa': 122,
 'ctf': 3,
 'rr': 2,
 'tdm': 2,
 'dom': 1}

In [56]:
# parse winrate and elo info
type_stats = soup.find('div', id = 'tab-{}'.format('overall')).text
type_stats = type_stats.split('\n')
type_stats = [x.strip() for x in type_stats if x.strip() != '']
type_stats

['Win Rate: 47.89% (3610 wins, 3928 losses)',
 'Kill Ratio:',
 '0.88 (68190 kills, 77496 deaths)',
 'Cap Ratio:',
 '-',
 'Best Rating: 1500 ± 300 (tdm, 0 games)',
 'Best B-Rating: TBD',
 'Best Rank:',
 '58 of 1485',
 '(ca, top 3.84%)',
 'Last Played:',
 '21:58:51',
 'Games Played:',
 '7538',
 'Favorite Map:',
 'campgrounds']

In [ ]:
soup.findAll('table', class_= 'table table-hover table-condensed')

[<table class="table table-hover table-condensed">
 <thead>
 <tr>
 <th>#</th>
 <th></th>
 <th>Played</th>
 <th>Type</th>
 <th>Server</th>
 <th>Map</th>
 <th>Result</th>
 <th>Opponent</th>
 <th>Rating</th>
 <th title="Rating ± Uncertainty">Old Glicko</th>
 <th title="Rating / Uncertainty">Glicko Change</th>
 </tr>
 </thead>
 <tbody>
 </tbody>
 </table>]

### Shut down the browser

In [ ]:
# driver.close()